In [1]:
# Ce notebook contient le code de l'application de machine learning développé dans le cadre de la détection de faux billets
# Le notebook est optimisé pour une lecture avec Voilà afin de rendre l'interface utiliateur plus conviviale.

In [2]:
#Importation des librairies
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import StringIO
import base64
import joblib
import os

In [3]:
# Chargement du modèle et du scaler
knn = joblib.load("modele/knn.pkl")
scaler = joblib.load("modele/scaler_knn.pkl")

In [4]:
# Widgets
upload_widget = widgets.FileUpload(accept='.csv', multiple=False)
bouton_prediction = widgets.Button(description="🔍 Prédire", button_style='info')
bouton_telechargement = widgets.Button(description="💾 Télécharger les prédictions", layout=widgets.Layout(width='auto'))
output_resultat = widgets.Output()
output_stats = widgets.Output()
output_camembert = widgets.Output()
output_export = widgets.Output()

In [5]:
# Variables globales (remplies lors de l'import)
df_resultats = pd.DataFrame()
nom_fichier_csv = ""

In [6]:
def traiter_fichier(change=None):
    global df_resultats, nom_fichier_csv
    if not upload_widget.value:
        return

    contenu = list(upload_widget.value.values())[0]['content']
    nom_fichier_csv = list(upload_widget.value.values())[0]['metadata']['name']
    df = pd.read_csv(StringIO(contenu.decode('utf-8')), sep=None, engine='python')

    # Prédictions
    ids = df['id']
    X = df[['margin_low', 'margin_up', 'length']]
    X_scaled = scaler.transform(X)
    y_pred = knn.predict(X_scaled)
    df['is_genuine'] = y_pred

    # Score de confiance basé sur les voisins (proportion de vote majoritaire)
    y_proba = knn.predict_proba(X_scaled)
    df['confidence'] = y_proba.max(axis=1)

    # Distance moyenne aux k voisins les plus proches
    distances, _ = knn.kneighbors(X_scaled)
    df['avg_distance'] = distances.mean(axis=1)

    # On copie le DataFrame une fois qu’il est enrichi
    df_resultats = df.copy()

    nb_vrai = (df_resultats['is_genuine'] == 1).sum()
    nb_faux = (df_resultats['is_genuine'] == 0).sum()
    ids_faux = df_resultats[df_resultats['is_genuine'] == 0]['id'].tolist()

    with output_resultat:
        clear_output()
        display(widgets.HTML("<h3 style='margin:0;'>Prédictions</h3>"))

        display(widgets.HTML(f"<div style='background:#51c8eb;padding:10px;margin-bottom:5px;border-radius:5px;'>✅ {nb_vrai} Vrais Billets identifiés</div>"))
        display(widgets.HTML(f"<div style='color:white;background:#902786;padding:10px;margin-bottom:10px;border-radius:5px;'>🛑 {nb_faux} Faux Billets identifiés</div>"))

        faux_ids_html = "".join([f"<button disabled style='margin:2px;'>{bid}</button>" for bid in ids_faux])
        display(widgets.HTML(f"<div style='background:#f0f0f0;padding:10px;border-radius:5px;margin-bottom:10px;'><b>Liste des billets identifiés comme faux :</b><br>{faux_ids_html}</div>"))
        
        
    with output_stats:
         # Légende explicative
        explication_html = """
        <div style='margin-top:10px; margin-bottom:10px;'>
            <b>Liste complète des billets et statistiques :</b><br>
            <ul style="font-size:10px;color:#666666;line-height:14px;">
                <li><b>is_genuine</b> : prédiction du modèle (1 = vrai billet, 0 = faux billet)</li>
                <li><b>confidence</b> : proportion de voisins ayant voté pour la classe prédite</li>
                <li><b>avg_distance</b> : distance moyenne aux voisins les plus proches (plus elle est faible, plus le billet est typique)</li>
            </ul>
        </div>
        """
        display(widgets.HTML(explication_html))

        # Affichage du DataFrame enrichi (avec arrondi propre)
        df_affichage = df_resultats.copy()
        df_affichage['confidence'] = df_affichage['confidence'].round(2)
        df_affichage['avg_distance'] = df_affichage['avg_distance'].round(4)
        display(df_affichage)

        display(bouton_telechargement)
        
    with output_camembert:
        clear_output()
        fig, ax = plt.subplots(figsize=(4, 4))
        sizes = [nb_vrai, nb_faux]
        labels = ["Vrai", "Faux"]
        colors = ['#51c8eb', '#902786']

        def make_label(pct, allvals):
            total = sum(sizes)
            val = int(round(pct * total / 100.0))
            return f"{'Vrai' if val == sizes[0] else 'Faux'}\n{pct:.1f}%"

        wedges, texts, autotexts = ax.pie(sizes, autopct=lambda pct: make_label(pct, sizes), colors=colors, textprops={'color':"white", 'fontsize': 10})
        ax.set_title("Répartition des vrais/faux billets", fontsize=11, weight='bold')
        plt.tight_layout()
        plt.show()

In [7]:
# Fonction de téléchargement
def telecharger_resultats(change):
    global df_resultats, nom_fichier_csv
    if not df_resultats.empty:
        nom_export = f"predictions_{nom_fichier_csv}"
        df_resultats.to_csv(nom_export, index=False)
        with output_export:
            output_export.clear_output()
            display(widgets.HTML(f"<b>✅ Fichier exporté : <code>{nom_export}</code></b>"))

In [8]:
# Association des boutons
bouton_prediction.on_click(traiter_fichier)
bouton_telechargement.on_click(telecharger_resultats)

In [9]:
# Affichage de l'interface

# Header avec boutons et titre prédiction
header = widgets.VBox([
    widgets.HTML("""
    <div style='background:#262766;color:white;padding:15px 20px;border-radius:10px;'>
        <h2 style='margin:0;'>Application de détection de faux billets</h2>
        <p style='margin:0;'>Uploader ici votre fichier CSV contenant les billets à vérifier. Le modèle prédira automatiquement s'ils sont vrais ou faux.</p>
    </div>
    """),
    widgets.HBox([upload_widget, bouton_prediction], layout=widgets.Layout(margin='10px 0')),
])

# Zone gauche : résultats + tableau
zone_gauche = widgets.VBox([
    output_resultat,
    output_export
], layout=widgets.Layout(width='70%', padding='10px'))

# Zone droite : camembert
zone_droite = widgets.VBox([
    output_camembert
], layout=widgets.Layout(width='30%', padding='20px 10px'))

# Assemblage en dashboard
corps = widgets.HBox([zone_gauche, zone_droite], layout=widgets.Layout(width='100%'))

# Affichage général
interface = widgets.VBox([header, corps, output_stats])
display(interface)